In [7]:
# Run these two blocks to load important libraries and set things up
import torch
from torch import nn
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

seed = 42

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True 


In [8]:
import torch
import os
from torch.utils.data import Dataset, DataLoader


class Seq2SeqDataset(Dataset):
    def __init__(self, data_dir):
        # store list of file paths
        self.file_paths = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith(".txt")]
        def note_to_key(note):
            notes = {
                "C": 1,
                "B#": 13,
                "C#": 2,
                "B##": 2,
                "Db": 2,
                "D": 3,
                "C##": 3,
                "D#": 4,
                "Eb": 4,
                "E": 5,
                "D##": 5,
                "Fb": 5,
                "F": 6,
                "E#": 6,
                "F#": 7,
                "E##": 7,
                "Gb": 7,
                "G": 8,
                "F##": 8,
                "G#": 9,
                "Ab": 9,
                "A": 10,
                "G##": 10,
                "A#": 11,
                "Bb": 11,
                "B": 12,
                "A##": 12,
                "Cb": -1
            }
            # B#5 = C6. C6 is 63 so octave = 5, note_name = B# = 13. 
            octave = int(note[-1])
            note_name = note[:-1]
            return int(((octave - 1) * 12) + notes[note_name] + 3)
        
        def make_label(lab):
            if len(lab) <= 2:
                return int(lab) + 5 if int(lab) < 0 else + 4# either left hand or right hand -1 or 1
            else: # else would be substitution 3_1 or -3_-1 ? or is it -3_1
                halves = lab.strip().split('_')
                return int(halves[0]) + 6 if int(halves[0]) < 0 else int(halves[0]) + 4
                
        # store input and output sequences for each file separately
        self.file_data = {}
        for file_path in self.file_paths:
            with open(file_path, 'r') as f:
                data = [line.strip().split('\t') for line in f]
                data = data[1:] # trim header
                input_seqs = [torch.tensor([note_to_key(x) for x in row[3].split()]) for row in data]
                output_seqs = [torch.tensor([make_label(x) for x in row[7].split()]) for row in data]
                self.file_data[file_path] = {'input': input_seqs, 'output': output_seqs}
    
        # compute the length of each file in the dataset
        self.file_lengths = {}
        for file_path in self.file_paths:
            file_len = len(self.file_data[file_path]['input'])
            self.file_lengths[file_path] = file_len - 9
        
        # compute the total length of the dataset
        self.total_length = sum(self.file_lengths.values())
    
    def __len__(self):
        return self.total_length
    
    def __getitem__(self, idx):
        # find the file that contains the item at the given index
        for file_path, file_len in self.file_lengths.items():
            if idx < file_len:
                break
            idx -= file_len
        
        # get input and output sequences from the file starting from the given index
        file_data = self.file_data[file_path]
        input_seq = torch.stack(file_data['input'][idx:idx+10])
        output_seq = torch.stack(file_data['output'][idx:idx+10])
        return input_seq, output_seq

In [9]:
from torch.utils.data.sampler import SubsetRandomSampler
ntotal = 60000
ntrain = int(0.9*ntotal)
nval = ntotal - ntrain

val_ix = np.random.choice(range(ntotal), size=nval, replace=False)
train_ix = list(set(range(ntotal)) - set(val_ix))

train_sampler = SubsetRandomSampler(train_ix)
val_sampler = SubsetRandomSampler(val_ix)

train_set = Seq2SeqDataset('./FingeringFiles')
test_set = Seq2SeqDataset('./FingeringFiles')

batch_size = 64
train_loader = DataLoader(train_set, batch_size, sampler=train_sampler)
val_loader = DataLoader(train_set, batch_size, sampler=val_sampler)
test_loader = DataLoader(test_set, batch_size)



FileNotFoundError: [Errno 2] No such file or directory: '../FingeringFiles'

In [ ]:
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)
        
    def forward(self, x, hc_0):
        # hc_0 needs to be a tuple of hidden and cell states, (h_0, c_0)
        lstm_outputs, hc_n = self.lstm(x.to(torch.float32), (hc_0[0].to(torch.float32), hc_0[1].to(torch.float32)))
        outputs = self.output_layer(lstm_outputs.to(torch.float32))
        
        return outputs, hc_n


In [ ]:
input_size = 1
hidden_size = 32
output_size = 10
LSTModel = MyLSTM(input_size, hidden_size, output_size) # 88 keys on a piano, ten fingers
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(LSTModel.parameters(), lr=0.00001)



In [ ]:
import torch.nn.functional as F

def train_network(model, train_loader, val_loader, criterion, optimizer, nepoch):
    try:
        for epoch in range(nepoch):
            print('EPOCH %d'%epoch)
            total_loss = 0
            count = 0
            hc_0 = (torch.zeros((1,10,hidden_size)), torch.zeros((1,10,hidden_size)))
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                # print(hc_0[0].detach().dtype)
                outputs, hc_0 = model(inputs, (hc_0[0].detach(), hc_0[1].detach()))
                # print(labels)
                labels = F.one_hot(labels, 10).squeeze()
                # print('slkjfklds', labels.shape, outputs.shape)
                # print(labels)
                
                # print(outputs[-1])
                loss = criterion(outputs[-1].to(torch.float32), labels.to(torch.float32))

                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                count += 1
            print('{:>12s} {:>7.5f}'.format('Train loss:', total_loss/count))
            with torch.no_grad():
                total_loss = 0
                count = 0
                for inputs, labels in val_loader:
                    outputs, hc_0 = model(inputs, (hc_0[0].detach(), hc_0[1].detach()))
                    labels = F.one_hot(labels, 10).squeeze()
                    loss = criterion(outputs[-1], labels)
                    total_loss += loss.item()
                    count += 1
                print('{:>12s} {:>7.5f}'.format('Val loss:', total_loss/count))
            print()
    except KeyboardInterrupt:
        print('Exiting from training early')
    return


In [ ]:
train_network(LSTModel, train_loader, val_loader, criterion, optimizer, nepoch=100)


EPOCH 0
 Train loss: 0.05045
   Val loss: 0.05075

EPOCH 1
 Train loss: 0.05045
   Val loss: 0.05077

EPOCH 2
 Train loss: 0.05044
   Val loss: 0.05075

EPOCH 3
 Train loss: 0.05045
   Val loss: 0.05076

EPOCH 4
 Train loss: 0.05044
   Val loss: 0.05077

EPOCH 5
 Train loss: 0.05044
   Val loss: 0.05075

EPOCH 6
 Train loss: 0.05044
   Val loss: 0.05075

EPOCH 7
 Train loss: 0.05044
   Val loss: 0.05076

EPOCH 8
 Train loss: 0.05043
   Val loss: 0.05075

EPOCH 9
 Train loss: 0.05043
   Val loss: 0.05076

EPOCH 10
 Train loss: 0.05043
   Val loss: 0.05074

EPOCH 11
 Train loss: 0.05042
   Val loss: 0.05075

EPOCH 12
 Train loss: 0.05042
   Val loss: 0.05074

EPOCH 13
 Train loss: 0.05041
   Val loss: 0.05074

EPOCH 14
 Train loss: 0.05042
   Val loss: 0.05075

EPOCH 15
 Train loss: 0.05041
   Val loss: 0.05072

EPOCH 16
 Train loss: 0.05041
   Val loss: 0.05074

EPOCH 17
 Train loss: 0.05041
   Val loss: 0.05073

EPOCH 18
 Train loss: 0.05040
   Val loss: 0.05072

EPOCH 19
 Train loss: 

In [ ]:
class MyBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyBiLSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, bidirectional=True)
        self.output_layer = torch.nn.Linear(hidden_size * 2, output_size)
        
    def forward(self, x, hc_0):
        # hc_0 needs to be a tuple of hidden and cell states, (h_0, c_0)
        lstm_outputs, hc_n = self.lstm(x.to(torch.float32), (hc_0[0].to(torch.float32), hc_0[1].to(torch.float32)))
        outputs = self.output_layer(lstm_outputs.to(torch.float32))
        
        return outputs, hc_n


In [ ]:
input_size = 1
hidden_size = 32
output_size = 10
BiLSTModel = MyBiLSTM(input_size * 2, hidden_size, output_size) # 88 keys on a piano, ten fingers
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(BiLSTModel.parameters(), lr=0.00001)



In [ ]:
import torch.nn.functional as F

def train_network2(model, train_loader, val_loader, criterion, optimizer, nepoch):
    try:
        for epoch in range(nepoch):
            print('EPOCH %d'%epoch)
            total_loss = 0
            count = 0
            hc_0 = (torch.zeros((2,10,hidden_size)), torch.zeros((2,10,hidden_size)))
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                # print(hc_0[0].detach().dtype)
                bi_dir = torch.cat([inputs, torch.flip(inputs, dims=[1])], dim=-1)
                # print(bi_dir.shape)
                outputs, hc_0 = model(bi_dir, (hc_0[0].detach(), hc_0[1].detach()))
                
                # print(labels)
                labels = F.one_hot(labels, 10).squeeze()
                # print('slkjfklds', labels.shape, outputs.shape)
                # print(labels)
                
                # print(outputs[-1])
                loss = criterion(outputs[-1].to(torch.float32), labels.to(torch.float32))

                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                count += 1
            print('{:>12s} {:>7.5f}'.format('Train loss:', total_loss/count))
            with torch.no_grad():
                total_loss = 0
                count = 0
                for inputs, labels in val_loader:
                    bi_dir = torch.cat([inputs, torch.flip(inputs, dims=[1])], dim=-1)
                    outputs, hc_0 = model(bi_dir, (hc_0[0].detach(), hc_0[1].detach()))
                    labels = F.one_hot(labels, 10).squeeze()
                    loss = criterion(outputs[-1], labels)
                    total_loss += loss.item()
                    count += 1
                print('{:>12s} {:>7.5f}'.format('Val loss:', total_loss/count))
            print()
    except KeyboardInterrupt:
        print('Exiting from training early')
    return


In [ ]:
train_network2(BiLSTModel, train_loader, val_loader, criterion, optimizer, nepoch=100)


EPOCH 0
 Train loss: 0.12022
   Val loss: 0.08551

EPOCH 1
 Train loss: 0.06875
   Val loss: 0.05777

EPOCH 2
 Train loss: 0.05410
   Val loss: 0.05257

EPOCH 3
 Train loss: 0.05179
   Val loss: 0.05161

EPOCH 4
 Train loss: 0.05117
   Val loss: 0.05128

EPOCH 5
 Train loss: 0.05086
   Val loss: 0.05110

EPOCH 6
 Train loss: 0.05073
   Val loss: 0.05100

EPOCH 7
 Train loss: 0.05067
   Val loss: 0.05094

EPOCH 8
 Train loss: 0.05064
   Val loss: 0.05091

EPOCH 9
 Train loss: 0.05061
   Val loss: 0.05090

EPOCH 10
 Train loss: 0.05058
   Val loss: 0.05088

EPOCH 11
 Train loss: 0.05056
   Val loss: 0.05085

EPOCH 12
 Train loss: 0.05055
   Val loss: 0.05086

EPOCH 13
 Train loss: 0.05054
   Val loss: 0.05083

EPOCH 14
 Train loss: 0.05053
   Val loss: 0.05082

EPOCH 15
 Train loss: 0.05052
   Val loss: 0.05081

EPOCH 16
 Train loss: 0.05052
   Val loss: 0.05084

EPOCH 17
 Train loss: 0.05051
   Val loss: 0.05080

EPOCH 18
 Train loss: 0.05051
   Val loss: 0.05081

EPOCH 19
 Train loss: 